In [26]:
#데이터 핸들링 라이브러리
import pandas as pd
import numpy as np
#데이터 시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams.update({'font.family':'AppleGothic'})
mpl.rc('axes', unicode_minus=False)
%config InlineBackend.figure_format = 'retina'

## 데이터 전처리 계획
1. 데이터 불러오기, 결측치 처리하기
2. 상대적으로 중요성이 떨어져보이는 피처 삭제, 필요하면 피처명을 직관적으로 다시 네이밍하기
3. 비숫자-문자열 독립변수 인코딩하기
4. 데이터 정규화
5. 훈련-테스트셋 분리

## 모델 학습 계획
1. 모델 생성
2. Cross_validation을 통한 모델학습
3. Accuracy, Precision,Recall, ROC 4개의 지표로 분류모델의 정확도 평가하기

In [27]:
#1. 데이터 불러오기 from TAAS: http://taas.koroad.or.kr/web/shp/sbm/initGisAnals.do?menuId=WEB_KMP_GIS_TAS
## 해당 경로에 있는 .csv 파일명 리스트 가져오기

import os
import time
t_0= time.time()

path = './교통사고_2021/'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.xls')] ## 파일명 끝이 .xls인 경우

## csv 파일들을 DataFrame으로 불러와서 concat

death_df = pd.DataFrame()
for i in file_list_py:
    data = pd.read_html(path + i)
    data= data[0]
    death_df = pd.concat([death_df,data])

death_df = death_df.reset_index(drop = True)
print(f'데이터 불러오기 및 통합에 걸린 시간은 {str(time.time()-t_0)} 입니다')
death_df

데이터 불러오기 및 통합에 걸린 시간은 32.162724018096924 입니다


,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
0,2021010100100022,2021년 1월 1일 02시,금요일,강원도 양양군 현남면,중상사고,0,1,1,0,차량단독 - 공작물충돌,...,맑음,단일로 - 기타,승용,여,19세,상해없음,NaN,NaN,NaN,NaN
1,2021010100100029,2021년 1월 1일 04시,금요일,강원도 횡성군 둔내면,경상사고,0,0,2,0,차량단독 - 공작물충돌,...,맑음,교차로 - 교차로안,승용,남,21세,경상,NaN,NaN,NaN,NaN
2,2021010100100040,2021년 1월 1일 08시,금요일,강원도 강릉시 강문동,중상사고,0,1,0,0,차대사람 - 횡단중,...,맑음,단일로 - 기타,화물,남,53세,상해없음,보행자,여,20세,중상
3,2021010100100065,2021년 1월 1일 11시,금요일,강원도 강릉시 옥계면,경상사고,0,0,4,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,승용,남,43세,상해없음,화물,남,49세,경상
4,2021010100100089,2021년 1월 1일 13시,금요일,강원도 동해시 천곡동,경상사고,0,0,2,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,62세,상해없음,승용,남,24세,경상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194089,2021123100100335,2021년 12월 31일 16시,금요일,충청남도 천안시 동남구 광덕면,경상사고,0,0,3,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,승용,여,32세,경상,승용,남,42세,경상
194090,2021123100100383,2021년 12월 31일 17시,금요일,충청남도 부여군 장암면,중상사고,0,1,0,0,차대차 - 추돌,...,맑음,단일로 - 기타,승용,남,24세,상해없음,자전거,남,78세,중상
194091,2021123100100384,2021년 12월 31일 17시,금요일,충청남도 천안시 동남구 청당동,경상사고,0,0,3,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,25세,경상,승용,남,22세,경상
194092,2021123100100578,2021년 12월 31일 21시,금요일,충청남도 보령시 명천동,경상사고,0,0,1,0,차대차 - 추돌,...,맑음,기타 - 기타,승용,남,59세,상해없음,승용,여,28세,경상


In [28]:
"""
결측치 확인하기 => 피해운전자의 차종,성별,연령,상해정도를 파악하지 못하는 데이터가 6687건이 존재함
원본데이터 194094에 비하면 삭제해도 학습에 영향력을 크게 미치지 못하뿐더러, 이러한 데이터들이
특정 편향성을 갖고 존재할 것이라 보기 어렵다(아마도 기록 누락에 의해 이러한 데이터가 존재할 것이기 때문)
그렇기 때문에 데이터를 제거해도 될것이라 판단하였음
"""
death_df.isnull().sum()
death_df.dropna(inplace=True)
death_df.isnull().sum()

사고번호          0
사고일시          0
요일            0
시군구           0
사고내용          0
사망자수          0
중상자수          0
경상자수          0
부상신고자수        0
사고유형          0
법규위반          0
노면상태          0
기상상태          0
도로형태          0
가해운전자 차종      0
가해운전자 성별      0
가해운전자 연령      0
가해운전자 상해정도    0
피해운전자 차종      0
피해운전자 성별      0
피해운전자 연령      0
피해운전자 상해정도    0
dtype: int64

In [29]:
#데이터 정보 찾기
print(death_df.info())
death_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187407 entries, 2 to 194093
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   사고번호        187407 non-null  int64 
 1   사고일시        187407 non-null  object
 2   요일          187407 non-null  object
 3   시군구         187407 non-null  object
 4   사고내용        187407 non-null  object
 5   사망자수        187407 non-null  int64 
 6   중상자수        187407 non-null  int64 
 7   경상자수        187407 non-null  int64 
 8   부상신고자수      187407 non-null  int64 
 9   사고유형        187407 non-null  object
 10  법규위반        187407 non-null  object
 11  노면상태        187407 non-null  object
 12  기상상태        187407 non-null  object
 13  도로형태        187407 non-null  object
 14  가해운전자 차종    187407 non-null  object
 15  가해운전자 성별    187407 non-null  object
 16  가해운전자 연령    187407 non-null  object
 17  가해운전자 상해정도  187407 non-null  object
 18  피해운전자 차종    187407 non-null  object
 19  피해운전자 성별    187407 non-

,사고번호,사고일시,요일,시군구,사고내용,사망자수,중상자수,경상자수,부상신고자수,사고유형,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
2,2021010100100040,2021년 1월 1일 08시,금요일,강원도 강릉시 강문동,중상사고,0,1,0,0,차대사람 - 횡단중,...,맑음,단일로 - 기타,화물,남,53세,상해없음,보행자,여,20세,중상
3,2021010100100065,2021년 1월 1일 11시,금요일,강원도 강릉시 옥계면,경상사고,0,0,4,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,승용,남,43세,상해없음,화물,남,49세,경상
4,2021010100100089,2021년 1월 1일 13시,금요일,강원도 동해시 천곡동,경상사고,0,0,2,0,차대차 - 측면충돌,...,맑음,단일로 - 기타,승용,남,62세,상해없음,승용,남,24세,경상
5,2021010100100123,2021년 1월 1일 15시,금요일,강원도 강릉시 포남동,사망사고,1,0,0,0,차대차 - 측면충돌,...,맑음,교차로 - 교차로안,원동기,남,16세,상해없음,승용,여,50세,상해없음
6,2021010100100146,2021년 1월 1일 17시,금요일,강원도 강릉시 옥천동,경상사고,0,0,2,0,차대차 - 추돌,...,맑음,교차로 - 교차로부근,승용,여,33세,상해없음,승용,남,27세,경상


In [30]:
death_df['사고내용'].value_counts()

경상사고    138186
중상사고     47072
사망사고      2149
Name: 사고내용, dtype: int64

In [31]:
#2.불필요해보이는 컬럼 제거
'''
사고번호는 각 사고에 대해 일대일 대응되는 key값으로 나중에 데이터를 결합시키는데 사용될 수 있어 보이므로 남겨둠
사망자수,중상자수,경상자수,부상신고자는 '피해운전자 상해정도'에 필요한 정보가 담겨있으므로 삭제한다
위치정보는 사용하지 않으므로 시군구도 삭제한다
성별도 유의미한 정보가 될수도 있으나 상대적인 중요도가 떨어지다고 판단되어 삭제한다
'''
death_df.drop(['시군구','사망자수','중상자수','경상자수','부상신고자수','노면상태','가해운전자 상해정도','피해운전자 상해정도','가해운전자 성별','피해운전자 성별'],axis=1,
              inplace=True)

## 컬럼 핸들링하기
##시간에 따른 시계가 교통사고의 요인중 하나일 수 있으므로 발생년월일시에서 시간만 떼어 새로운 컬럼으로 만듬
time_lst= list(death_df['사고일시'])
hour_lst=[]
for time in time_lst:
    hour_lst.append((str(time)[-3:-1]))
death_df['사고시각']= hour_lst
#사고일시 컬럼은 삭제
death_df.drop(['사고일시'],axis=1,inplace=True)


In [32]:
#피해운전자, 가해운전자 연령 미분류 삭제하기
idx= death_df[death_df['피해운전자 연령']=='미분류'].index
death_df.drop(idx,inplace=True)
idx= death_df[death_df['피해운전자 연령']=='98세 이상'].index
death_df.drop(idx,inplace=True)

idx2= death_df[death_df['가해운전자 연령']=='미분류'].index
death_df.drop(idx2,inplace=True)


#나이 컬럼: 숫자단위만 뽑기
suspect_lst= list(death_df['가해운전자 연령'])
suspect_old= []
for old in suspect_lst:
    suspect_old.append(str((old))[:-1])

death_df['가해운전자 연령']=suspect_old

victim_lst= list(death_df['피해운전자 연령'])
victim_old= []
for old in victim_lst:
    victim_old.append(str((old))[:-1])
death_df['피해운전자 연령']=victim_old


In [33]:
death_df=death_df.reindex(columns=['사고시각','사고유형','법규위반','기상상태','도로형태','가해운전자 차종','가해운전자 연령','피해운전자 차종','피해운전자 연령','사고번호','사고내용'])

In [34]:
#데이터 자료형 변환: 문자열-> 정수형
# death_df['사고시각']=death_df['사고시각'].astype('int')
# death_df['가해운전자 연령']=death_df['가해운전자 연령'].astype('int')
# death_df['피해운전자 연령']=death_df['피해운전자 연령'].astype('int')

![](https://scikit-learn.org/stable/_images/multi_org_chart.png)

In [35]:
#3.범주형 변수 인코딩하기:원 핫 인코딩을 하도록 한다
#한번만 인코딩해야됨. 두번하면 안됨
# from sklearn.preprocessing import LabelEncoder,OneHotEncoder
#
# categories_backup=[]
# features=['사고시각','사고유형','법규위반','기상상태','도로형태','가해운전자 차종','피해운전자 차종','사고내용', '가해운전자 연령','피해운전자 연령']
# features= np.array(features).reshape(-1,1)
# for feature in features:
#     OHE= OneHotEncoder()
#     OHE.fit(death_df[feature])
#     #카테고리 백업해두기
#     categories_backup.append(OHE.categories_)
#     #인코딩하기
#     death_df[feature]= OHE.transform(death_df[feature])
# death_df

In [36]:
features=['사고시각','사고유형','법규위반','기상상태','도로형태','가해운전자 차종','피해운전자 차종','사고내용', '가해운전자 연령','피해운전자 연령']
encoded_df= pd.get_dummies(data=death_df,columns=features,prefix=features)

In [37]:
encoded_df

,사고번호,사고시각_00,사고시각_01,사고시각_02,사고시각_03,사고시각_04,사고시각_05,사고시각_06,사고시각_07,사고시각_08,...,피해운전자 연령_89,피해운전자 연령_9,피해운전자 연령_90,피해운전자 연령_91,피해운전자 연령_92,피해운전자 연령_93,피해운전자 연령_94,피해운전자 연령_95,피해운전자 연령_96,피해운전자 연령_97
2,2021010100100040,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,2021010100100065,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021010100100089,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2021010100100123,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2021010100100146,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194089,2021123100100335,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
194090,2021123100100383,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
194091,2021123100100384,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
194092,2021123100100578,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
#독립변수와 종속변수로 분리하기
y= 0* encoded_df['사고내용_경상사고'] + 1* encoded_df['사고내용_중상사고'] + 2*encoded_df['사고내용_사망사고']
X= encoded_df.drop(['사고번호','사고내용_경상사고','사고내용_중상사고','사고내용_사망사고'],axis=1)

#데이터 분할하기
from sklearn.model_selection import train_test_split

#전체 데이터중 70퍼센트를 학습에 30퍼센트를 테스트 데이터로 분리한다
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [51]:
import optuna  # pip install optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

def objective(trial, X, y):
    param_grid = {}  # to be filled in later
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = lgbm.LGBMClassifier(objective="multiclass", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="multi_logloss",
            early_stopping_rounds=100,
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = preds

    return np.mean(cv_scores)



In [50]:
conda install optuna

Solving environment: done

## Package Plan ##

  environment location: /Users/wooseongkyun/miniforge3

  added / updated specs:
    - optuna


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alembic-1.8.1              |     pyhd8ed1ab_0         137 KB  conda-forge
    autopage-0.5.1             |     pyhd8ed1ab_0          26 KB  conda-forge
    cliff-3.10.1               |     pyhd8ed1ab_0          53 KB  conda-forge
    cmaes-0.8.2                |     pyh44b312d_0          16 KB  conda-forge
    cmd2-2.4.2                 |   py39h2804cbe_0         211 KB  conda-forge
    colorlog-6.6.0             |   py39h2804cbe_1          17 KB  conda-forge
    greenlet-1.1.2             |   py39h0ef5a74_2          87 KB  conda-forge
    mako-1.2.1                 |     pyhd8ed1ab_0          61 KB  conda-forge
    optuna-2.10.1              |     pyhd8ed1ab_0         179 KB  conda-forge
    pbr-5.9

In [45]:
# from lightgbm import LGBMClassifier
# from sklearn.metrics import roc_auc_score
# from hyperopt import fmin,tpe, Trials
# #위의 학습 데이터중 90퍼센트를 학습 데이터에, 10퍼센트는 검증용 데이터로 분리한다
# X_tr,X_val,y_tr,y_val= train_test_split(X_train,y_train,test_size=0.1)
#
# #하이퍼 파라미터 튜닝하기
# from hyperopt import hp
# lgbm_search_space={
#                    'objective':'multiclass',
#                    'metric' : 'multi_logloss',
#                    'num_leaves':hp.quniform('num_leaves',32,64,1),
#                    'max_depth': hp.quniform('max_depth',100,160,1),
#                    'min_child_samples':hp.quniform('min_child_samples',60,100,1),
#                    'subsample':hp.uniform('subsample',0.7,1),
#                    'learning_rate':hp.uniform('learning_rate',0.01,0.2)
#                    }
#
# from sklearn.model_selection import KFold
# from hyperopt import fmin,tpe,Trials
# import random
#
# def objective_fun(search_space):
#     lgbm_clf= LGBMClassifier(
#
#         n_estimators=100,
#         objective='multiclass',
#         num_leaves=int(search_space['num_leaves']),
#         max_depth=int(search_space['max_depth']),
#         min_child_samples=int(search_space['min_child_samples']),
#         subsample= search_space['subsample'],
#         learning_rate=search_space['learning_rate']
#     )
#     #3개의 k-fold 방식으로 평가된 rou_auc를 담는 list
#     roc_auc_lst =[]
#
#     #3개의 k-fold 방식을 적용하기
#     kf= KFold(n_splits=3)
#     #X_train을 다시 학습과 검증용 데이터로 분리한다
#     for tr_idx, val_idx in kf.split(X_train):
#         #kf.split(X_train)으로 추출된 학습과 검증 idx값으로 학습과 검증 데이터 셋 분리
#         X_tr,y_tr= X_train.iloc[tr_idx],y_train.iloc[tr_idx]
#         X_val,y_val=X_train.iloc[val_idx], y_train.iloc[val_idx]
#
#         #early stopping은 100회로 설정하고 추출된 학습과 검증 데이터로 lgbm_classfier를 수행한다
#         lgbm_clf.fit(X_tr,y_tr,early_stopping_rounds=30,eval_metric='auc',eval_set=[(X_tr,y_tr),(X_val,y_val)])
#
#         #1로 예측한 확률값 추출 후 roc_auc를 계산하고 평균 roc_auc를 계산하기 위해 list에 결과값을 담는다
#         score= roc_auc_score(y_val,lgbm_clf.predict_proba(X_val)[:,1])
#         roc_auc_lst.append(score)
#
#     #3개의 K-fold값으로 계산된 roc_auc값의 평균값을 반환하되
#     #Hyperopt는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환한다
#     return -1 * np.mean(roc_auc_lst)
#
# trials= Trials()
# #fmin() 함수를 호출, max_evals를 지정된 횟수만큼 반복 후 목적함수의 최솟값을 찾는  최적입력값을 추출한다
# best= fmin(fn=objective_fun,space=lgbm_search_space,algo=tpe.suggest,
#            trials=trials,rstate=np.random.default_rng(seed=50),max_evals=50)
#
#


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/wooseongkyun/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

[LightGBM] [Fatal] Multiclass objective and metrics don't match
job exception: Multiclass objective and metrics don't match



  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]


LightGBMError: Multiclass objective and metrics don't match

In [ ]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,roc_auc_score
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
#
#
# #독립변수와 종속변수로 분리하기
# y= death_df['사고내용']
# X= death_df.drop(['사고번호','사고내용','가해운전자 성별','피해운전자 성별'],axis=1)
#
# #데이터 분할하기
# from sklearn.model_selection import train_test_split
# #classification용 모델 라이브러리 가져오기
# from lightgbm import LGBMClassifier
#
# #전체 데이터중 70퍼센트를 학습에 30퍼센트를 테스트 데이터로 분리한다
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
#
# params={
#     'max_depth':[8,16,24],
#     'min_samples_leaf':[1,6,12],
#     'min_samples_split':[2,6,16]
# }
#
# #RandomForestCLassifier 객체 생성후 GridSearchCV 수행
# rf_clf= RandomForestClassifier(n_estimators=100,random_state=0,n_jobs=-1)
# grid_cv= GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
# grid_cv.fit(X_train,y_train)
#
# print('최적 하이퍼 파라미터는 :',grid_cv.best_params_)
# print('최고 예측 정확도는',grid_cv.best_score_)

In [ ]:
# #하이퍼 파라미터 튜닝하기
# from hyperopt import hp
# lgbm_search_space={'num_leaves':hp.quniform('num_leaves',32,64,1),
#                    'max_depth': hp.quniform('max_depth',100,160,1),
#                    'min_child_samples':hp.quniform('min_child_samples',60,100,1),
#                    'subsample':hp.uniform('subsample',0.7,1),
#                    'learning_rate':hp.uniform('learning_rate',0.01,0.2)
#                    }


In [ ]:
# from sklearn.model_selection import KFold
# from hyperopt import fmin,tpe,Trials
# import random
#
# def objective_fun(search_space):
#     lgbm_clf= LGBMClassifier(
#                              class_weight='balanced',
#                              n_estimators=100,
#                              num_leaves=int(search_space['num_leaves']),
#                              max_depth=int(search_space['max_depth']),
#                              min_child_samples=int(search_space['min_child_samples']),
#                              subsample= search_space['subsample'],
#                              learning_rate=search_space['learning_rate']
#                              )
#     #3개의 k-fold 방식으로 평가된 rou_auc를 담는 list
#     roc_auc_lst =[]
#
#     #3개의 k-fold 방식을 적용하기
#     kf= KFold(n_splits=3)
#     #X_train을 다시 학습과 검증용 데이터로 분리한다
#     for tr_idx, val_idx in kf.split(X_train):
#         #kf.split(X_train)으로 추출된 학습과 검증 idx값으로 학습과 검증 데이터 셋 분리
#         X_tr,y_tr= X_train.iloc[tr_idx],y_train.iloc[tr_idx]
#         X_val,y_val=X_train.iloc[val_idx], y_train.iloc[val_idx]
#
#         #early stopping은 100회로 설정하고 추출된 학습과 검증 데이터로 lgbm_classfier를 수행한다
#         lgbm_clf.fit(X_tr,y_tr,early_stopping_rounds=30,eval_metric='auc',eval_set=[(X_tr,y_tr),(X_val,y_val)])
#
#         #1로 예측한 확률값 추출 후 roc_auc를 계산하고 평균 roc_auc를 계산하기 위해 list에 결과값을 담는다
#         score= roc_auc_score(y_val,lgbm_clf.predict_proba(X_val)[:,1])
#         roc_auc_lst.append(score)
#
#     #3개의 K-fold값으로 계산된 roc_auc값의 평균값을 반환하되
#     #Hyperopt는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환한다
#     return -1 * np.mean(roc_auc_lst)


In [ ]:
# trials= Trials()
# #fmin() 함수를 호출, max_evals를 지정된 횟수만큼 반복 후 목적함수의 최솟값을 찾는  최적입력값을 추출한다
# best= fmin(fn=objective_fun,space=lgbm_search_space,algo=tpe.suggest,
#            trials=trials,rstate=np.random.default_rng(seed=50),max_evals=50)
#


In [ ]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,roc_auc_score
#
#
# print('오차 행렬 값은: \n',confusion_matrix(y_test,y_pred))
# print(f'accuracy 스코어는 {accuracy_score(y_test,y_pred):.5f} 입니다')
# print(f"precision 스코어는 {precision_score(y_test,y_pred,average='macro'):.5f}")
# print(f"recall 스코어는 {recall_score(y_test,y_pred,average='macro'):.5f}")
